## 1. Tensorboard

In [ ]:
import torch
from torch import nn, optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt
import time

# --- Step 1: Synthetic Data ---
x = torch.linspace(-5, 5, 100).unsqueeze(1)
y = 3 * x + 0.8 * torch.randn(x.size())

# --- Step 2: Define two different models ---
model_A = nn.Sequential(nn.Linear(1, 1))                     
model_B = nn.Sequential(nn.Linear(1, 10), nn.ReLU(), nn.Linear(10, 1))  

criterion = nn.MSELoss()
optimizer_A = optim.SGD(model_A.parameters(), lr=0.01)
optimizer_B = optim.Adam(model_B.parameters(), lr=0.001)

writer_A = SummaryWriter(log_dir="runs/model_A")
writer_B = SummaryWriter(log_dir="runs/model_B")

epochs = 100

# --- Step 3: Training loop ---
for epoch in range(epochs):
    # ===== Model A =====
    optimizer_A.zero_grad()
    outputs_A = model_A(x)
    loss_A = criterion(outputs_A, y)
    loss_A.backward()
    optimizer_A.step()

    # Log scalar metrics
    writer_A.add_scalar("Loss/train", loss_A.item(), epoch)
    writer_A.add_scalar("LR", optimizer_A.param_groups[0]['lr'], epoch)

    # Log histograms for weights, biases, and gradients
    for name, param in model_A.named_parameters():
        writer_A.add_histogram(f"{name}/weights", param, epoch)
        if param.grad is not None:
            writer_A.add_histogram(f"{name}/grads", param.grad, epoch)

    # ===== Model B =====
    optimizer_B.zero_grad()
    outputs_B = model_B(x)
    loss_B = criterion(outputs_B, y)
    loss_B.backward()
    optimizer_B.step()

    writer_B.add_scalar("Loss/train", loss_B.item(), epoch)
    writer_B.add_scalar("LR", optimizer_B.param_groups[0]['lr'], epoch)

    for name, param in model_B.named_parameters():
        writer_B.add_histogram(f"{name}/weights", param, epoch)
        if param.grad is not None:
            writer_B.add_histogram(f"{name}/grads", param.grad, epoch)

    # --- Combined loss (for easy comparison in one chart) ---
    writer_A.add_scalars("Comparison/Loss", {"Model_A": loss_A.item(), "Model_B": loss_B.item()}, epoch)

    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d}: Loss A = {loss_A.item():.4f}, Loss B = {loss_B.item():.4f}")

# --- Step 4: Visualize model predictions ---
with torch.no_grad():
    pred_A = model_A(x)
    pred_B = model_B(x)

plt.figure(figsize=(6,4))
plt.scatter(x.numpy(), y.numpy(), label="True Data", color="gray")
plt.plot(x.numpy(), pred_A.numpy(), label="Model A (Linear)", color="blue")
plt.plot(x.numpy(), pred_B.numpy(), label="Model B (Deep)", color="red")
plt.legend()
plt.title("Model Predictions")
plt.savefig("predictions.png")

# Log the comparison plot as image
image = np.moveaxis(plt.imread("predictions.png"), -1, 0)
writer_A.add_image("Predictions", image)
writer_B.add_image("Predictions", image)

# --- Step 5: Flush and close writers ---
writer_A.flush()
writer_B.flush()
writer_A.close()
writer_B.close()
time.sleep(1)

print("Training complete. Use TensorBoard to visualize the logs.")
print("Run: tensorboard --logdir=runs")

In [ ]:
# --- Install wandb if not installed ---
# pip install wandb torch matplotlib

import torch
from torch import nn, optim
import wandb
import matplotlib.pyplot as plt

# --- Step 1: Synthetic Data ---
x = torch.linspace(-5, 5, 100).unsqueeze(1)
y = 3 * x + 0.8 * torch.randn(x.size())

# --- Step 2: Define two models ---
model_A = nn.Sequential(nn.Linear(1, 1))                     # simple linear
model_B = nn.Sequential(nn.Linear(1, 10), nn.ReLU(), nn.Linear(10, 1))  # slightly deeper

criterion = nn.MSELoss()
optimizer_A = optim.SGD(model_A.parameters(), lr=0.01)
optimizer_B = optim.Adam(model_B.parameters(), lr=0.01)

# --- Step 3: Initialize W&B project ---
wandb.init(project="two-models", name="Combined_Run")

# Optional: Watch models to log weights and gradients
wandb.watch(model_A, log="all", log_freq=10)
wandb.watch(model_B, log="all", log_freq=10)

epochs = 100

# --- Step 4: Training Loop ---
for epoch in range(epochs):
    # ---- Train Model A ----
    optimizer_A.zero_grad()
    out_A = model_A(x)
    loss_A = criterion(out_A, y)
    loss_A.backward()
    optimizer_A.step()

    # ---- Train Model B ----
    optimizer_B.zero_grad()
    out_B = model_B(x)
    loss_B = criterion(out_B, y)
    loss_B.backward()
    optimizer_B.step()

    # ---- Log metrics for both models ----
    wandb.log({
        "epoch": epoch,
        "Loss/Model_A": loss_A.item(),
        "Loss/Model_B": loss_B.item()
    })

    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d} | Loss A: {loss_A.item():.4f} | Loss B: {loss_B.item():.4f}")

# --- Step 5: Log prediction plots ---
with torch.no_grad():
    pred_A = model_A(x).detach().numpy()
    pred_B = model_B(x).detach().numpy()

plt.figure(figsize=(6,4))
plt.scatter(x.numpy(), y.numpy(), label="True Data", color="gray")
plt.plot(x.numpy(), pred_A, label="Model A Prediction", color="blue")
plt.plot(x.numpy(), pred_B, label="Model B Prediction", color="red")
plt.legend()
plt.title("Model Predictions")
plt.savefig("predictions.png")
plt.close()

# Log the plot to W&B
wandb.log({"Predictions": wandb.Image("predictions.png")})

# --- Step 6: Finish W&B run ---
wandb.finish()

print("Training complete! You can view your run on W&B online dashboard.")


Epoch 000 | Loss A: 41.1978 | Loss B: 78.0126
Epoch 010 | Loss A: 1.7731 | Loss B: 62.0151
Epoch 020 | Loss A: 0.8247 | Loss B: 47.5393
Epoch 030 | Loss A: 0.8018 | Loss B: 32.7020
Epoch 040 | Loss A: 0.8012 | Loss B: 18.1579
Epoch 050 | Loss A: 0.8012 | Loss B: 7.2090
Epoch 060 | Loss A: 0.8011 | Loss B: 2.3166
Epoch 070 | Loss A: 0.8011 | Loss B: 1.2696
Epoch 080 | Loss A: 0.8011 | Loss B: 1.1545
Epoch 090 | Loss A: 0.8011 | Loss B: 1.1152


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Loss/Model_A,█▆▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/Model_B,█▇▇▇▇▆▅▅▅▄▄▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇██
Loss/Model_A,0.80111
Loss/Model_B,1.01966
epoch,99


✅ Training complete! You can view your run on W&B online dashboard.


In [7]:
import torch
from torch import nn, optim
import mlflow
import mlflow.pytorch
import matplotlib.pyplot as plt
import os

# --- Windows-friendly paths ---
MLFLOW_RUNS_PATH = "D:\\The-Office\\Substack\\experiment-tracking"  # change if needed
os.makedirs(MLFLOW_RUNS_PATH, exist_ok=True)
mlflow.set_tracking_uri(f"file:///{MLFLOW_RUNS_PATH}")
mlflow.set_experiment("Two_Model_Comparison")

# --- Synthetic Data ---
x = torch.linspace(-5, 5, 100).unsqueeze(1)
y = 3 * x + 0.8 * torch.randn(x.size())

# --- Models ---
model_A = nn.Sequential(nn.Linear(1, 1))
model_B = nn.Sequential(nn.Linear(1, 10), nn.ReLU(), nn.Linear(10, 1))

criterion = nn.MSELoss()
optimizer_A = optim.SGD(model_A.parameters(), lr=0.01)
optimizer_B = optim.Adam(model_B.parameters(), lr=0.01)

epochs = 100

# --- Train & log model function ---
def train_model(model, optimizer, model_name):
    with mlflow.start_run(run_name=model_name):
        # Log hyperparameters
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("optimizer", optimizer.__class__.__name__)
        mlflow.log_param("learning_rate", optimizer.param_groups[0]['lr'])
        mlflow.log_param("epochs", epochs)

        for epoch in range(epochs):
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            # Log loss per epoch
            mlflow.log_metric("train_loss", loss.item(), step=epoch)

            if epoch % 10 == 0:
                print(f"{model_name} | Epoch {epoch:03d} | Loss: {loss.item():.4f}")

        # --- Log model with input example (convert tensor to numpy) ---
        mlflow.pytorch.log_model(model, name="model", input_example=x[:5].numpy())

        # --- Log prediction plot ---
        with torch.no_grad():
            preds = model(x).detach().numpy()
        plt.figure(figsize=(6,4))
        plt.scatter(x.numpy(), y.numpy(), label="True Data", color="gray")
        plt.plot(x.numpy(), preds, label=f"{model_name} Prediction", color="orange")
        plt.legend()
        plt.title(f"{model_name} Predictions")
        plot_path = f"{model_name}_predictions.png"
        plt.savefig(plot_path)
        plt.close()
        mlflow.log_artifact(plot_path)

# --- Train both models ---
train_model(model_A, optimizer_A, "Model_A")
train_model(model_B, optimizer_B, "Model_B")

print("\nTraining complete! Start MLflow UI:")
print(f"python -m mlflow ui --backend-store-uri file:///{MLFLOW_RUNS_PATH}")
print("Open http://127.0.0.1:5000 in your browser.")

Model_A | Epoch 000 | Loss: 101.1440
Model_A | Epoch 010 | Loss: 3.3405
Model_A | Epoch 020 | Loss: 0.8533
Model_A | Epoch 030 | Loss: 0.7037
Model_A | Epoch 040 | Loss: 0.6402
Model_A | Epoch 050 | Loss: 0.5986
Model_A | Epoch 060 | Loss: 0.5709
Model_A | Epoch 070 | Loss: 0.5524
Model_A | Epoch 080 | Loss: 0.5400
Model_A | Epoch 090 | Loss: 0.5318


2025/10/12 14:02:18 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Model_B | Epoch 000 | Loss: 104.0675
Model_B | Epoch 010 | Loss: 84.1900
Model_B | Epoch 020 | Loss: 70.6771
Model_B | Epoch 030 | Loss: 59.6342
Model_B | Epoch 040 | Loss: 48.2134
Model_B | Epoch 050 | Loss: 35.3818
Model_B | Epoch 060 | Loss: 22.2125
Model_B | Epoch 070 | Loss: 11.0660
Model_B | Epoch 080 | Loss: 4.1188
Model_B | Epoch 090 | Loss: 1.5975


2025/10/12 14:02:30 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



Training complete! Start MLflow UI:
python -m mlflow ui --backend-store-uri file:///D:\The-Office\Substack\experiment-tracking
Open http://127.0.0.1:5000 in your browser.
